In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from hmmlearn import hmm
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ======== CONFIGURABLE PARAMETERS ========
# Market data parameters
TICKER = 'SPY'  # Main ticker to analyze
VIX_TICKER = '^VIX'  # Volatility index
TNX_TICKER = '^TNX'  # 10-Year Treasury Yield
GLD_TICKER = 'GLD'  # Gold ETF
XLY_TICKER = 'XLY'  # Consumer Discretionary ETF
XLP_TICKER = 'XLP'  # Consumer Staples ETF
XLU_TICKER = 'XLU'  # Utilities ETF
XLF_TICKER = 'XLF'  # Financial ETF
HYG_TICKER = 'HYG'  # High Yield Corporate Bond ETF
TLT_TICKER = 'TLT'  # 20+ Year Treasury Bond ETF
VIX3M_TICKER = '^VIX3M'  # 3-Month VIX
IRX_TICKER = '^IRX'  # 2-Year Treasury Yield
UUP_TICKER = 'UUP'  # US Dollar Index ETF
TIP_TICKER = 'TIP'  # TIPS ETF
IEF_TICKER = 'IEF'  # 7-10 Year Treasury ETF
START_DATE = "1995-01-01"  # Historical data start date

# HMM model parameters  
NUM_REGIMES = 3  # Number of market regimes
MAX_ITERATIONS = 1000  # Max iterations for HMM convergence
COVARIANCE_TYPE = 'full'  # Options: 'full', 'tied', 'diag', 'spherical'

# Technical indicator parameters (keeping same as original)
VOL_WINDOW = 21  # Window for volatility calculation (21 days ~ 1 month)
MOMENTUM_WINDOW = 63  # Window for momentum calculation (63 days ~ 3 months)
SMA_FAST = 20  # Fast moving average
SMA_SLOW = 50  # Slow moving average
BB_WINDOW = 20  # Bollinger Bands window
BB_STD = 2  # Bollinger Bands standard deviation multiplier
ATR_WINDOW = 14  # Average True Range window
CHOP_WINDOW = 14  # Choppiness Index window
SECTOR_WINDOW = 10  # Window for sector rotation indicators
CREDIT_MA_WINDOW = 30  # Window for credit spread MA

# Training period
TRAIN_START_DATE = "2018-01-05"
TRAIN_END_DATE = "2025-01-01"

# ======== DATA PREPARATION FUNCTIONS ========

def load_ad_line_data(filepath="nyse_breadth_2023.csv"):
    """Load and prepare NYSE breadth data for A/D line indicators"""
    print(f"Loading A/D line data from {filepath}...")
    
    # Read the CSV file
    ad_data = pd.read_csv(filepath)
    
    # Convert the date column to datetime
    ad_data['Date'] = pd.to_datetime(ad_data['Date'])
    
    # Calculate basic breadth metrics
    # 1. Cumulative A/D Line
    ad_data['Net_Advances'] = ad_data['Advancers'] - ad_data['Decliners']
    ad_data['AD_Line'] = ad_data['Net_Advances'].cumsum()
    
    # 2. A/D Ratio and Z-Score
    ad_data['AD_Ratio'] = ad_data['Advancers'] / ad_data['Decliners']
    ad_data['Log_AD_Ratio'] = np.log(ad_data['AD_Ratio'])
    ad_data['Log_AD_Ratio_21d_Mean'] = ad_data['Log_AD_Ratio'].rolling(window=21).mean()
    ad_data['Log_AD_Ratio_21d_StdDev'] = ad_data['Log_AD_Ratio'].rolling(window=21).std()
    ad_data['AD_Ratio_Z_Score'] = (ad_data['Log_AD_Ratio'] - ad_data['Log_AD_Ratio_21d_Mean']) / ad_data['Log_AD_Ratio_21d_StdDev']
    
    # 3. McClellan Oscillator
    # Calculate EMAs of Net Advances
    ad_data['EMA19_Net_Advances'] = ad_data['Net_Advances'].ewm(span=19, adjust=False).mean()
    ad_data['EMA39_Net_Advances'] = ad_data['Net_Advances'].ewm(span=39, adjust=False).mean()
    ad_data['McClellan_Oscillator'] = ad_data['EMA19_Net_Advances'] - ad_data['EMA39_Net_Advances']
    # Normalize by total issues
    ad_data['Total_Issues'] = ad_data['Advancers'] + ad_data['Decliners'] + ad_data['Neutral']
    ad_data['McClellan_Oscillator_Norm'] = ad_data['McClellan_Oscillator'] / (ad_data['Total_Issues']) * 1000
    
    # 4. Percentage of Advancing Issues
    ad_data['Advancing_Percentage'] = ad_data['Advancers'] / ad_data['Total_Issues'] * 100
    ad_data['Advancing_Percentage_10MA'] = ad_data['Advancing_Percentage'].rolling(window=10).mean()
    ad_data['Advancing_Percentage_Z'] = (ad_data['Advancing_Percentage'] - ad_data['Advancing_Percentage'].rolling(window=21).mean()) / ad_data['Advancing_Percentage'].rolling(window=21).std()
    
    # 5. Breadth Thrust Indicator
    ad_data['Daily_Thrust'] = ad_data['Advancers'] / (ad_data['Advancers'] + ad_data['Decliners'])
    ad_data['Breadth_Thrust'] = ad_data['Daily_Thrust'].ewm(span=10, adjust=False).mean()
    ad_data['Thrust_Signal'] = (ad_data['Breadth_Thrust'] > 0.65).astype(int)
    
    # 6. McClellan Summation Index
    ad_data['McClellan_Summation_Index'] = ad_data['McClellan_Oscillator'].cumsum()
    ad_data['McClellan_SI_10MA'] = ad_data['McClellan_Summation_Index'].rolling(window=10).mean()
    
    # 7. A/D Line Momentum
    ad_data['AD_Line_5d_ROC'] = ad_data['AD_Line'].pct_change(periods=5) * 100
    ad_data['AD_Line_10d_ROC'] = ad_data['AD_Line'].pct_change(periods=10) * 100
    ad_data['AD_Line_20d_ROC'] = ad_data['AD_Line'].pct_change(periods=20) * 100
    ad_data['AD_Line_Momentum_Z'] = (ad_data['AD_Line_10d_ROC'] - ad_data['AD_Line_10d_ROC'].rolling(window=50).mean()) / ad_data['AD_Line_10d_ROC'].rolling(window=50).std()
    
    # 8. A/D Line Moving Average Crossovers
    ad_data['AD_Line_20MA'] = ad_data['AD_Line'].rolling(window=20).mean()
    ad_data['AD_Line_50MA'] = ad_data['AD_Line'].rolling(window=50).mean()
    ad_data['AD_Line_Golden_Cross'] = (ad_data['AD_Line_20MA'] > ad_data['AD_Line_50MA']).astype(int)
    ad_data['AD_Line_Dist_20MA'] = (ad_data['AD_Line'] / ad_data['AD_Line_20MA'] - 1) * 100
    
    # 9. Composite Breadth Indicator
    # Normalize each component to range [0,1]
    ad_data['AD_Ratio_Z_Norm'] = (ad_data['AD_Ratio_Z_Score'].clip(-3, 3) + 3) / 6
    ad_data['McClellan_Osc_Norm'] = (ad_data['McClellan_Oscillator'].clip(-150, 150) + 150) / 300
    ad_data['Breadth_Thrust_Norm'] = ad_data['Breadth_Thrust']
    ad_data['Advancing_Pct_Norm'] = ad_data['Advancing_Percentage'] / 100
    
    # Create composite
    ad_data['Composite_Breadth'] = (ad_data['AD_Ratio_Z_Norm'] + 
                                   ad_data['McClellan_Osc_Norm'] + 
                                   ad_data['Breadth_Thrust_Norm'] + 
                                   ad_data['Advancing_Pct_Norm']) / 4
    
    ad_data['Composite_Breadth_Z'] = (ad_data['Composite_Breadth'] - 
                                     ad_data['Composite_Breadth'].rolling(window=50).mean()) / ad_data['Composite_Breadth'].rolling(window=50).std().fillna(1)
    
    return ad_data

def download_market_data(ticker, vix_ticker, tnx_ticker, gld_ticker, xly_ticker, xlp_ticker, xlu_ticker, xlf_ticker, hyg_ticker, tlt_ticker, start_date):
    """Download and prepare market data"""
    end_date = (datetime.today() + timedelta(days=1)).strftime("%Y-%m-%d")
    print(f"Downloading market data from {start_date} to {end_date}...")
    
    # Download ticker, VIX, TNX, GLD, sector ETFs, and bond ETF data
    df_ticker = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)
    df_vix = yf.download(vix_ticker, start=start_date, end=end_date)
    df_tnx = yf.download(tnx_ticker, start=start_date, end=end_date)
    df_gld = yf.download(gld_ticker, start=start_date, end=end_date)
    df_xly = yf.download(xly_ticker, start=start_date, end=end_date)
    df_xlp = yf.download(xlp_ticker, start=start_date, end=end_date)
    df_xlu = yf.download(xlu_ticker, start=start_date, end=end_date)
    df_xlf = yf.download(xlf_ticker, start=start_date, end=end_date)
    df_hyg = yf.download(hyg_ticker, start=start_date, end=end_date)
    df_tlt = yf.download(tlt_ticker, start=start_date, end=end_date)
    
    # Download additional tickers for new features
    df_vix3m = yf.download(VIX3M_TICKER, start=start_date, end=end_date)
    df_irx = yf.download(IRX_TICKER, start=start_date, end=end_date)
    df_uup = yf.download(UUP_TICKER, start=start_date, end=end_date)
    df_tip = yf.download(TIP_TICKER, start=start_date, end=end_date)
    df_ief = yf.download(IEF_TICKER, start=start_date, end=end_date)
    
    # Fix column structure and reset index
    if len(df_ticker.columns.names) > 1:
        df_ticker.columns = df_ticker.columns.droplevel(1)
    if len(df_vix.columns.names) > 1:
        df_vix.columns = df_vix.columns.droplevel(1)
    if len(df_tnx.columns.names) > 1:
        df_tnx.columns = df_tnx.columns.droplevel(1)
    if len(df_gld.columns.names) > 1:
        df_gld.columns = df_gld.columns.droplevel(1)
    if len(df_xly.columns.names) > 1:
        df_xly.columns = df_xly.columns.droplevel(1)
    if len(df_xlp.columns.names) > 1:
        df_xlp.columns = df_xlp.columns.droplevel(1)
    if len(df_xlu.columns.names) > 1:
        df_xlu.columns = df_xlu.columns.droplevel(1)
    if len(df_xlf.columns.names) > 1:
        df_xlf.columns = df_xlf.columns.droplevel(1)
    if len(df_hyg.columns.names) > 1:
        df_hyg.columns = df_hyg.columns.droplevel(1)
    if len(df_tlt.columns.names) > 1:
        df_tlt.columns = df_tlt.columns.droplevel(1)
    
    # Fix column structure for new tickers
    if len(df_vix3m.columns.names) > 1:
        df_vix3m.columns = df_vix3m.columns.droplevel(1)
    if len(df_irx.columns.names) > 1:
        df_irx.columns = df_irx.columns.droplevel(1)
    if len(df_uup.columns.names) > 1:
        df_uup.columns = df_uup.columns.droplevel(1)
    if len(df_tip.columns.names) > 1:
        df_tip.columns = df_tip.columns.droplevel(1)
    if len(df_ief.columns.names) > 1:
        df_ief.columns = df_ief.columns.droplevel(1)
    
    df_ticker = df_ticker.reset_index()
    df_vix = df_vix.reset_index()
    df_tnx = df_tnx.reset_index()
    df_gld = df_gld.reset_index()
    df_xly = df_xly.reset_index()
    df_xlp = df_xlp.reset_index()
    df_xlu = df_xlu.reset_index()
    df_xlf = df_xlf.reset_index()
    df_hyg = df_hyg.reset_index()
    df_tlt = df_tlt.reset_index()
    
    df_vix3m = df_vix3m.reset_index()
    df_irx = df_irx.reset_index()
    df_uup = df_uup.reset_index()
    df_tip = df_tip.reset_index()
    df_ief = df_ief.reset_index()
    
    # Keep only Date and Close from VIX
    df_vix = df_vix[['Date', 'Close']].rename(columns={'Close': 'VIX'})
    
    # Keep only Date and Close from TNX
    df_tnx = df_tnx[['Date', 'Close']].rename(columns={'Close': 'TNX'})
    
    # Keep only relevant columns from GLD, sector ETFs, and bond ETFs
    df_gld = df_gld[['Date', 'Close']].rename(columns={'Close': 'GLD'})
    df_xly = df_xly[['Date', 'Close']].rename(columns={'Close': 'XLY'})
    df_xlp = df_xlp[['Date', 'Close']].rename(columns={'Close': 'XLP'})
    df_xlu = df_xlu[['Date', 'Close']].rename(columns={'Close': 'XLU'})
    df_xlf = df_xlf[['Date', 'Close']].rename(columns={'Close': 'XLF'})
    df_hyg = df_hyg[['Date', 'Close']].rename(columns={'Close': 'HYG'})
    df_tlt = df_tlt[['Date', 'Close']].rename(columns={'Close': 'TLT'})
    
    # Keep only relevant columns from new tickers
    df_vix3m = df_vix3m[['Date', 'Close']].rename(columns={'Close': 'VIX3M'})
    df_irx = df_irx[['Date', 'Close']].rename(columns={'Close': 'IRX'})
    df_uup = df_uup[['Date', 'Close']].rename(columns={'Close': 'UUP'})
    df_tip = df_tip[['Date', 'Close']].rename(columns={'Close': 'TIP'})
    df_ief = df_ief[['Date', 'Close']].rename(columns={'Close': 'IEF'})
    
    # Merge data
    df = pd.merge(df_ticker, df_vix, on='Date', how='left')
    df = pd.merge(df, df_tnx, on='Date', how='left')
    df = pd.merge(df, df_gld, on='Date', how='left')
    df = pd.merge(df, df_xly, on='Date', how='left')
    df = pd.merge(df, df_xlp, on='Date', how='left')
    df = pd.merge(df, df_xlu, on='Date', how='left')
    df = pd.merge(df, df_xlf, on='Date', how='left')
    df = pd.merge(df, df_hyg, on='Date', how='left')
    df = pd.merge(df, df_tlt, on='Date', how='left')
    
    # Merge new tickers
    df = pd.merge(df, df_vix3m, on='Date', how='left')
    df = pd.merge(df, df_irx, on='Date', how='left')
    df = pd.merge(df, df_uup, on='Date', how='left')
    df = pd.merge(df, df_tip, on='Date', how='left')
    df = pd.merge(df, df_ief, on='Date', how='left')
    
    # Fill missing values with forward fill method
    df['VIX'] = df['VIX'].fillna(method='ffill')
    df['TNX'] = df['TNX'].fillna(method='ffill')
    df['GLD'] = df['GLD'].fillna(method='ffill')
    df['XLY'] = df['XLY'].fillna(method='ffill')
    df['XLP'] = df['XLP'].fillna(method='ffill')
    df['XLU'] = df['XLU'].fillna(method='ffill')
    df['XLF'] = df['XLF'].fillna(method='ffill')
    df['HYG'] = df['HYG'].fillna(method='ffill')
    df['TLT'] = df['TLT'].fillna(method='ffill')
    
    # Fill missing values for new tickers
    df['VIX3M'] = df['VIX3M'].fillna(method='ffill')
    df['IRX'] = df['IRX'].fillna(method='ffill')
    df['UUP'] = df['UUP'].fillna(method='ffill')
    df['TIP'] = df['TIP'].fillna(method='ffill')
    df['IEF'] = df['IEF'].fillna(method='ffill')
    
    # Calculate daily log returns
    df['LogVIX'] = np.log(df['VIX'])
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1)) * 100
    df['GLD_Log_Return'] = np.log(df['GLD'] / df['GLD'].shift(1)) * 100
    
    return df

def calculate_bollinger_bands(df, window=20, num_std=2):
    """Calculate Bollinger Bands and related metrics"""
    # Calculate Bollinger Bands
    df['BB_Middle'] = df['Close'].rolling(window=window).mean()
    rolling_std = df['Close'].rolling(window=window).std()
    df['BB_Upper'] = df['BB_Middle'] + (rolling_std * num_std)
    df['BB_Lower'] = df['BB_Middle'] - (rolling_std * num_std)
    
    # Calculate Bollinger Band Width (normalized)
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle'] * 100
    
    # Calculate %B (position within the bands)
    df['BB_PercentB'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    
    return df

def calculate_atr(df, window=14):
    """Calculate Average True Range (ATR)"""
    # Calculate True Range
    df['TR1'] = abs(df['High'] - df['Low'])
    df['TR2'] = abs(df['High'] - df['Close'].shift(1))
    df['TR3'] = abs(df['Low'] - df['Close'].shift(1))
    df['True_Range'] = df[['TR1', 'TR2', 'TR3']].max(axis=1)
    
    # Calculate ATR using Wilder's smoothing method
    df['ATR'] = df['True_Range'].rolling(window=window).mean()
    
    # Normalize ATR by price
    df['ATR_Normalized'] = df['ATR'] / df['Close'] * 100
    
    # Clean up intermediate columns
    df = df.drop(['TR1', 'TR2', 'TR3'], axis=1)
    
    return df

def calculate_choppiness_index(df, window=14):
    """Calculate Choppiness Index"""
    if 'ATR' not in df.columns:
        df = calculate_atr(df, window)
    
    df['MaxHi'] = df['High'].rolling(window=window).max()
    df['MinLo'] = df['Low'].rolling(window=window).min()
    df['ATR_Sum'] = df['ATR'].rolling(window=window).sum()
    
    # Calculate Choppiness Index
    df['Choppiness_Index'] = 100 * np.log10(df['ATR_Sum'] / (df['MaxHi'] - df['MinLo'])) / np.log10(window)
    
    # Clean up intermediate columns
    df = df.drop(['MaxHi', 'MinLo', 'ATR_Sum'], axis=1)
    
    return df

def calculate_features(data, ad_data=None):
    """Calculate features for regime classification, now with A/D line indicators"""
    df = data.copy()
    
    # Volatility features
    df['Volatility'] = df['Log_Return'].rolling(window=VOL_WINDOW).std() * np.sqrt(252)  # Annualized
    df['Volume_Z_Score'] = (df['Volume'] - df['Volume'].rolling(window=VOL_WINDOW).mean()) / df['Volume'].rolling(window=VOL_WINDOW).std()
    
    # Trend features
    df['Momentum'] = df['Close'].pct_change(periods=MOMENTUM_WINDOW) * 100
    df['SMA_Fast'] = df['Close'].rolling(window=SMA_FAST).mean()
    df['SMA_Slow'] = df['Close'].rolling(window=SMA_SLOW).mean()
    df['SMA_Ratio'] = df['SMA_Fast'] / df['SMA_Slow']
    
    # Price distance from moving averages
    df['Price_to_SMA_Fast'] = df['Close'] / df['SMA_Fast'] - 1
    df['Price_to_SMA_Slow'] = df['Close'] / df['SMA_Slow'] - 1
    
    # VIX-based features
    df['VIX_Ratio'] = df['VIX'] / df['VIX'].rolling(window=VOL_WINDOW).mean()
    df['VIX_Change'] = df['VIX'].pct_change(periods=5) * 100  # 5-day VIX change
    
    # Treasury Yield (TNX) features
    df['TNX_Level'] = df['TNX']  # Absolute yield level
    df['TNX_Daily_Change'] = df['TNX'].diff() * 100  # Daily change in basis points
    df['TNX_Weekly_Change'] = df['TNX'].diff(5) * 100  # 1-week change in basis points
    df['TNX_Z_Score'] = (df['TNX'] - df['TNX'].rolling(window=VOL_WINDOW).mean()) / df['TNX'].rolling(window=VOL_WINDOW).std()
    df['TNX_Ratio'] = df['TNX'] / df['TNX'].rolling(window=VOL_WINDOW).mean()
    
    # Gold (GLD) features
    if df['GLD'].notna().any():
        df['GLD_Volatility'] = df['GLD_Log_Return'].rolling(window=VOL_WINDOW).std() * np.sqrt(252)
        df['GLD_SPY_Ratio'] = df['GLD'] / df['Close']
        df['GLD_SPY_Ratio_Change'] = df['GLD_SPY_Ratio'].pct_change(periods=5) * 100
        df['GLD_Z_Score'] = (df['GLD'] - df['GLD'].rolling(window=VOL_WINDOW).mean()) / df['GLD'].rolling(window=VOL_WINDOW).std()
        df['GLD_Momentum'] = df['GLD'].pct_change(periods=MOMENTUM_WINDOW) * 100
        df['GLD_SPY_Momentum_Diff'] = df['GLD_Momentum'] - df['Momentum']
    
    # Sector Rotation indicators
    if df['XLY'].notna().any() and df['XLP'].notna().any():
        df['XLY_XLP_Ratio'] = df['XLY'] / df['XLP']
        df['XLY_XLP_Change'] = df['XLY_XLP_Ratio'].pct_change(periods=SECTOR_WINDOW) * 100
        df['XLY_XLP_Z'] = (df['XLY_XLP_Ratio'] - df['XLY_XLP_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['XLY_XLP_Ratio'].rolling(window=VOL_WINDOW).std()
    
    if df['XLU'].notna().any():
        df['XLU_SPY_Ratio'] = df['XLU'] / df['Close']
        df['XLU_SPY_Change'] = df['XLU_SPY_Ratio'].pct_change(periods=SECTOR_WINDOW) * 100
        df['XLU_SPY_Z'] = (df['XLU_SPY_Ratio'] - df['XLU_SPY_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['XLU_SPY_Ratio'].rolling(window=VOL_WINDOW).std()
    
    if df['XLF'].notna().any():
        df['XLF_SPY_Ratio'] = df['XLF'] / df['Close']
        df['XLF_SPY_Change'] = df['XLF_SPY_Ratio'].pct_change(periods=SECTOR_WINDOW) * 100
        df['XLF_SPY_Z'] = (df['XLF_SPY_Ratio'] - df['XLF_SPY_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['XLF_SPY_Ratio'].rolling(window=VOL_WINDOW).std()
    
    # Credit Spread indicators (HYG/TLT)
    if df['HYG'].notna().any() and df['TLT'].notna().any():
        df['HYG_TLT_Ratio'] = df['HYG'] / df['TLT']
        df['HYG_TLT_Daily_Change'] = df['HYG_TLT_Ratio'].pct_change() * 100
        df['HYG_TLT_MA'] = df['HYG_TLT_Ratio'].rolling(window=CREDIT_MA_WINDOW).mean()
        df['HYG_TLT_MA_Diff'] = (df['HYG_TLT_Ratio'] / df['HYG_TLT_MA'] - 1) * 100
        df['HYG_TLT_Z'] = (df['HYG_TLT_Ratio'] - df['HYG_TLT_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['HYG_TLT_Ratio'].rolling(window=VOL_WINDOW).std()
        df['HYG_TLT_Volatility'] = df['HYG_TLT_Daily_Change'].rolling(window=VOL_WINDOW).std()
    
    # Volume features
    df['Volume_Trend'] = df['Volume'] / df['Volume'].rolling(window=VOL_WINDOW).mean()
    df['Volume_to_Volatility'] = df['Volume'] / (df['Volatility'] + 1e-10)
    
    # Combined features
    df['Return_Volatility_Ratio'] = df['Log_Return'] / (df['Volatility'] + 1e-10)
    df['VIX_Volatility_Ratio'] = df['VIX'] / (df['Volatility'] + 1e-10)
    
    # Add Bollinger Bands features
    df = calculate_bollinger_bands(df, window=BB_WINDOW, num_std=BB_STD)
    
    # Add ATR features
    df = calculate_atr(df, window=ATR_WINDOW)
    
    # Add Choppiness Index
    df = calculate_choppiness_index(df, window=CHOP_WINDOW)
    
    # Calculate Z-scores for the new indicators
    df['BB_Width_Z'] = (df['BB_Width'] - df['BB_Width'].rolling(window=VOL_WINDOW).mean()) / df['BB_Width'].rolling(window=VOL_WINDOW).std()
    df['ATR_Norm_Z'] = (df['ATR_Normalized'] - df['ATR_Normalized'].rolling(window=VOL_WINDOW).mean()) / df['ATR_Normalized'].rolling(window=VOL_WINDOW).std()
    df['Choppiness_Z'] = (df['Choppiness_Index'] - df['Choppiness_Index'].rolling(window=VOL_WINDOW).mean()) / df['Choppiness_Index'].rolling(window=VOL_WINDOW).std()
    
    # Add the new features requested by the user
    
    # 1. Yield Curve Spread (10Y-2Y)
    if 'TNX' in df.columns and 'IRX' in df.columns:
        # Calculate the spread in percentage points
        df['Yield_Curve_Spread'] = df['TNX'] - df['IRX']
        
        # Calculate Z-score of the spread
        df['Yield_Curve_Spread_Z'] = (df['Yield_Curve_Spread'] - df['Yield_Curve_Spread'].rolling(window=VOL_WINDOW).mean()) / df['Yield_Curve_Spread'].rolling(window=VOL_WINDOW).std()
        
        # Calculate weekly change in the spread
        df['Yield_Curve_Spread_Change'] = df['Yield_Curve_Spread'].diff(5) * 100  # 1-week change in basis points
        
        # Calculate inversion flag (1 if inverted, 0 if normal)
        df['Yield_Curve_Inverted'] = (df['Yield_Curve_Spread'] < 0).astype(int)
    
    # 2. VIX/VIX3M Ratio (Volatility Term Structure)
    if 'VIX' in df.columns and 'VIX3M' in df.columns:
        # Calculate ratio (values < 1 indicate normal contango, values > 1 indicate backwardation/fear)
        df['VIX_VIX3M_Ratio'] = df['VIX'] / df['VIX3M']
        
        # Calculate Z-score of the ratio
        df['VIX_VIX3M_Ratio_Z'] = (df['VIX_VIX3M_Ratio'] - df['VIX_VIX3M_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['VIX_VIX3M_Ratio'].rolling(window=VOL_WINDOW).std()
        
        # Calculate spread (for alternative representation)
        df['VIX_VIX3M_Spread'] = df['VIX'] - df['VIX3M']
        
        # Flag for backwardation (1 if in backwardation, 0 if in contango)
        df['VIX_Backwardation'] = (df['VIX_VIX3M_Ratio'] > 1).astype(int)
    
    # 3. UUP Z-Score (Dollar Strength)
    if 'UUP' in df.columns:
        # Calculate UUP returns
        df['UUP_Return'] = df['UUP'].pct_change() * 100
        
        # Calculate UUP Z-score (normalized dollar strength)
        df['UUP_Z_Score'] = (df['UUP'] - df['UUP'].rolling(window=VOL_WINDOW).mean()) / df['UUP'].rolling(window=VOL_WINDOW).std()
        
        # Calculate UUP momentum
        df['UUP_Momentum'] = df['UUP'].pct_change(periods=VOL_WINDOW) * 100
        
        # Calculate UUP volatility
        df['UUP_Volatility'] = df['UUP_Return'].rolling(window=VOL_WINDOW).std() * np.sqrt(252)
    
    # 4. TIP/IEF Ratio (Inflation Expectations)
    if 'TIP' in df.columns and 'IEF' in df.columns:
        # Calculate the ratio
        df['TIP_IEF_Ratio'] = df['TIP'] / df['IEF']
        
        # Calculate Z-score of the ratio
        df['TIP_IEF_Ratio_Z'] = (df['TIP_IEF_Ratio'] - df['TIP_IEF_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['TIP_IEF_Ratio'].rolling(window=VOL_WINDOW).std()
        
        # Calculate weekly change in the ratio
        df['TIP_IEF_Ratio_Change'] = df['TIP_IEF_Ratio'].pct_change(periods=5) * 100  # 5-day change
        
        # Calculate TIP performance vs SPY
        df['TIP_SPY_Ratio'] = df['TIP'] / df['Close']
        df['TIP_SPY_Ratio_Z'] = (df['TIP_SPY_Ratio'] - df['TIP_SPY_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['TIP_SPY_Ratio'].rolling(window=VOL_WINDOW).std()
    
    # ADD NEW A/D LINE FEATURES
    if ad_data is not None:
        # Merge A/D line data with price data
        ad_data_subset = ad_data[['Date', 'AD_Line', 'AD_Ratio_Z_Score', 'McClellan_Oscillator', 
                                 'McClellan_Oscillator_Norm', 'Advancing_Percentage_Z',
                                 'Breadth_Thrust', 'McClellan_Summation_Index',
                                 'AD_Line_Momentum_Z', 'AD_Line_Dist_20MA', 
                                 'Composite_Breadth_Z', 'AD_Line_Golden_Cross']]
        
        # Convert to datetime to ensure proper merging
        df['Date'] = pd.to_datetime(df['Date'])
        ad_data_subset['Date'] = pd.to_datetime(ad_data_subset['Date'])
        
        # Merge on Date
        df = pd.merge(df, ad_data_subset, on='Date', how='left')
        
        # Calculate AD Line Divergence with Price
        if 'AD_Line' in df.columns:
            # Calculate rates of change for AD Line and Price
            df['AD_Line_20d_ROC'] = df['AD_Line'].pct_change(periods=20) * 100
            df['Price_20d_ROC'] = df['Close'].pct_change(periods=20) * 100
            
            # Calculate divergence spread
            df['AD_Price_Divergence'] = df['AD_Line_20d_ROC'] - df['Price_20d_ROC']
            
            # Calculate correlation over 30-day window
            df['AD_Price_Correlation'] = df['AD_Line'].rolling(window=30).corr(df['Close'])
            
            # Flag negative divergences (AD_Line not confirming price highs)
            price_new_high = df['Close'] > df['Close'].rolling(window=20).max().shift(1)
            ad_line_not_confirming = df['AD_Line'] < df['AD_Line'].rolling(window=20).max().shift(1)
            df['AD_Negative_Divergence'] = (price_new_high & ad_line_not_confirming).astype(int)
    
    # Fill NaN values and drop remaining NaN rows
    df = df.dropna().reset_index(drop=True)
    
    return df

# ======== FEATURE SELECTION & DIMENSIONALITY REDUCTION ========
def select_and_prepare_features(data, n_components=5):
    """
    Select key features and reduce dimensionality using PCA
    
    This step is critical for HMM as it works better with fewer dimensions
    """
    # Updated feature selection based on requested changes plus new A/D line features
    selected_features = [
        'Log_Return',              # Daily returns
        'Volatility',              # Recent volatility
        'Price_to_SMA_Fast',       # Short-term trend
        'Price_to_SMA_Slow',       # Long-term trend
        'Momentum',                # Medium-term price momentum
        'VIX_Ratio',               # VIX relative to its recent average (kept - removed VIX_Z_Score)
        'Volume_Z_Score',          # Volume relative to its recent history
        'BB_Width_Z',              # Normalized Bollinger Band width
        'ATR_Norm_Z',              # Normalized ATR
        'Choppiness_Z',            # Market choppiness/trendiness
        'TNX_Z_Score',             # 10-Year yield relative to recent history
        'GLD_SPY_Ratio_Change',    # Gold/SPY ratio change
        'XLY_XLP_Z',               # Discretionary vs. Staples (kept - removed XLP_SPY_Z)
        'HYG_TLT_Z',               # Credit spread
        'XLP_SPY_Z',               # Staples vs. SPY
        'Yield_Curve_Spread_Z',    # NEW: Yield curve spread (10Y-2Y) Z-score
        'VIX_VIX3M_Ratio',         # NEW: VIX/VIX3M ratio
        'UUP_Z_Score',             # NEW: UUP (Dollar) strength Z-score
        'TIP_IEF_Ratio_Z',         # NEW: TIP/IEF (inflation expectations) Z-score
        # NEW A/D LINE FEATURES
        'AD_Ratio_Z_Score',        # A/D Ratio Z-Score
        'McClellan_Oscillator_Norm', # Normalized McClellan Oscillator
        'Advancing_Percentage_Z',  # Z-Score of % Advancing Issues
        'Breadth_Thrust',          # Breadth Thrust Indicator
        'AD_Line_Momentum_Z',      # Z-Score of A/D Line momentum
        'AD_Price_Divergence',     # Divergence between A/D Line and Price
        'Composite_Breadth_Z'      # Z-Score of Composite Breadth Indicator
    ]
    
    # Check if any feature has all NaN values and remove it
    valid_features = []
    for feature in selected_features:
        if feature in data.columns and not data[feature].isna().all():
            valid_features.append(feature)
        elif feature in data.columns:
            print(f"Warning: Feature '{feature}' contains all NaN values and will be excluded")
        else:
            print(f"Warning: Feature '{feature}' not found in dataset")
    
    selected_features = valid_features
    feature_data = data[selected_features].values
    
    # Standardize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(feature_data)
    
    # Reduce dimensionality with PCA
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(scaled_features)
    
    # Print explained variance
    explained_variance = pca.explained_variance_ratio_
    cum_explained_variance = np.cumsum(explained_variance)
    print(f"PCA explained variance: {explained_variance}")
    print(f"Cumulative explained variance: {cum_explained_variance}")
    
    # Print feature importance in each component
    print("\nPCA Component Loadings:")
    component_df = pd.DataFrame(
        pca.components_.T,
        columns=[f'PC{i+1}' for i in range(n_components)],
        index=selected_features
    )
    print(component_df.abs().sort_values(by='PC1', ascending=False))
    
    return reduced_features, scaler, pca, selected_features

# ======== HMM MODEL FUNCTIONS ========
def train_hmm_model(data, start_date, end_date, n_components=3, max_iter=1000, cov_type='full'):
    """Train HMM model on selected features with proper initialization"""
    # Filter data to training period
    training = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)].copy()
    
    print(f"Training HMM model on data from {start_date} to {end_date}")
    print(f"Training data shape: {training.shape}")
    
    # Reduce dimensionality 
    # Use fewer components (3-5) as HMM works better with lower dimensions
    reduced_features, scaler, pca, selected_features = select_and_prepare_features(training, n_components=4)
    
    # Initialize HMM with domain knowledge
    # Key innovation: Use informed starting values rather than random
    
    # Initialize transition matrix with high self-transition probabilities
    # This enforces regime persistence
    transition_matrix = np.array([
        [0.97, 0.02, 0.01],  # 97% chance to stay in bull
        [0.03, 0.95, 0.02],  # 95% chance to stay in neutral
        [0.02, 0.03, 0.95]   # 95% chance to stay in bear
    ])
    
    # Initialize starting probabilities - assume we start in bull market
    startprob = np.array([0.7, 0.2, 0.1])
    
    # Initialize means for each regime
    # Component 0: Bull regime (positive returns, low volatility)
    # Component 1: Neutral regime (flat returns, moderate volatility)
    # Component 2: Bear regime (negative returns, high volatility)
    
    # First calculate simple statistics to guide initialization
    # Use k-means or simple threshold-based approach to set initial means
    from sklearn.cluster import KMeans
    
    kmeans = KMeans(n_clusters=n_components, random_state=42)
    cluster_labels = kmeans.fit_predict(reduced_features)
    
    # Calculate mean returns for each cluster
    mean_returns = []
    for i in range(n_components):
        cluster_mask = (cluster_labels == i)
        if np.sum(cluster_mask) > 0:
            mean_return = np.mean(training.loc[cluster_mask, 'Log_Return'])
            mean_returns.append((i, mean_return))
    
    # Sort clusters by mean returns
    mean_returns.sort(key=lambda x: x[1], reverse=True)
    
    # Map from sorted returns to regime index
    cluster_to_regime = {}
    regime_labels = ['Bull', 'Neutral', 'Bear']
    
    # Assign highest returns to Bull, lowest to Bear, middle to Neutral
    for idx, (cluster, _) in enumerate(mean_returns):
        if idx == 0:
            cluster_to_regime[cluster] = 0  # Bull
        elif idx == len(mean_returns) - 1:
            cluster_to_regime[cluster] = 2  # Bear
        else:
            cluster_to_regime[cluster] = 1  # Neutral
    
    # Calculate initial means from kmeans
    initial_means = np.zeros((n_components, reduced_features.shape[1]))
    for i in range(n_components):
        for cluster, regime in cluster_to_regime.items():
            if regime == i:
                mask = (cluster_labels == cluster)
                if np.sum(mask) > 0:
                    initial_means[i] = np.mean(reduced_features[mask], axis=0)
    
    # Create the HMM model
    model = hmm.GaussianHMM(
        n_components=n_components,
        covariance_type=cov_type,
        n_iter=max_iter,
        tol=1e-6,
        init_params="",  # Don't initialize params, we'll set them manually
        random_state=42
    )
    
    # Set initial parameters based on domain knowledge
    model.startprob_ = startprob
    model.transmat_ = transition_matrix
    model.means_ = initial_means
    
    # For covariances, we'll initialize with the empirical covariances of the clusters
    if cov_type in ['full', 'tied']:
        covs = []
        for i in range(n_components):
            for cluster, regime in cluster_to_regime.items():
                if regime == i:
                    mask = (cluster_labels == cluster)
                    if np.sum(mask) > 0:
                        cov = np.cov(reduced_features[mask].T)
                        # Ensure numerical stability
                        cov += np.eye(cov.shape[0]) * 1e-6
                        covs.append(cov)
        
        if cov_type == 'full':
            model.covars_ = np.array(covs)
        else:  # tied
            model.covars_ = np.mean(covs, axis=0)
    
    # Fit the model
    model.fit(reduced_features)
    
    # Get state sequence and state probabilities using the Forward-Backward algorithm
    hidden_states = model.predict(reduced_features)
    state_probs = model.predict_proba(reduced_features)
    
    # Extract more detailed state probabilities using the Forward-Backward algorithm
    # This gives smoother probabilities than just predict_proba
    _, forwback = model.score_samples(reduced_features)
    
    # Map states to regime labels based on behavior
    # We'll analyze the characteristics of each state to determine labels
    regime_stats = {}
    regime_labels = [""] * n_components
    
    # Analyze the characteristics of each regime
    for i in range(n_components):
        regime_mask = (hidden_states == i)
        if np.sum(regime_mask) > 0:
            regime_stats[i] = {
                'count': np.sum(regime_mask),
                'return_avg': np.mean(training.loc[regime_mask, 'Log_Return']),
                'return_std': np.std(training.loc[regime_mask, 'Log_Return']),
                'volatility_avg': np.mean(training.loc[regime_mask, 'Volatility']),
                'vix_avg': np.mean(training.loc[regime_mask, 'VIX']),
                'vix_ratio_avg': np.mean(training.loc[regime_mask, 'VIX_Ratio']),
                'momentum_avg': np.mean(training.loc[regime_mask, 'Momentum']),
                'prob_avg': np.mean(state_probs[:, i])
            }
            
            # Add more stats if available
            for feature in ['BB_Width', 'ATR_Normalized', 'Choppiness_Index', 'TNX', 
                           'GLD_SPY_Ratio', 'XLY_XLP_Ratio', 'HYG_TLT_Ratio']:
                if feature in training.columns:
                    regime_stats[i][f'{feature.lower()}_avg'] = np.mean(training.loc[regime_mask, feature])
    
    # Determine regime labels based on average returns and volatility
    # This is a more robust method than just using the state number
    return_avgs = [regime_stats[i]['return_avg'] if i in regime_stats else 0 for i in range(n_components)]
    volatility_avgs = [regime_stats[i]['volatility_avg'] if i in regime_stats else 0 for i in range(n_components)]
    
    # Assign labels: highest returns = Bull, lowest returns = Bear, middle = Neutral
    # We also consider volatility as a secondary factor
    state_characteristics = [(i, return_avgs[i], volatility_avgs[i]) for i in range(n_components)]
    
    # Sort by returns (highest to lowest)
    state_characteristics.sort(key=lambda x: x[1], reverse=True)
    
    # Assign labels
    for rank, (state, _, _) in enumerate(state_characteristics):
        if rank == 0:
            regime_labels[state] = 'Bull'
        elif rank == n_components - 1:
            regime_labels[state] = 'Bear'
        else:
            regime_labels[state] = 'Neutral'
    
    # Print key regime statistics
    print("\nRegime Characteristics Summary:")
    print("=" * 80)
    print(f"{'Regime':<8} {'Label':<8} {'Count':<8} {'Return %':<10} {'Vol':<8} {'VIX':<8} {'Momentum':<10}")
    print("-" * 80)
    
    for i in range(n_components):
        if i in regime_stats:
            stats = regime_stats[i]
            print(f"{i:<8} {regime_labels[i]:<8} {stats['count']:<8} "
                  f"{stats['return_avg']:<10.2f} {stats['volatility_avg']:<8.2f} {stats['vix_avg']:<8.2f} "
                  f"{stats['momentum_avg']:<10.2f}")
    
    # Calculate transition probability matrix from the Markov model
    transition_matrix = model.transmat_
    
    # Print transition matrix
    print("\nRegime Transition Matrix (daily probabilities):")    
    print("=" * 60)
    print(f"{'From/To':<10}", end="")
    for i in range(n_components):
        print(f"{regime_labels[i]:<10}", end="")
    print()
    print("-" * 60)
    
    for i in range(n_components):
        print(f"{regime_labels[i]:<10}", end="")
        for j in range(n_components):
            print(f"{transition_matrix[i, j]:<10.4f}", end="")
        print()
    
    # Calculate expected regime duration in days
    print("\nExpected Regime Duration:")
    for i in range(n_components):
        duration = 1 / (1 - transition_matrix[i, i]) if transition_matrix[i, i] < 1 else float('inf')
        print(f"{regime_labels[i]:<10} {duration:<10.1f} days")
    
    # Calculate AIC and BIC for model evaluation
    print(f"\nModel Evaluation:")
    # Calculate number of parameters
    n_states = model.n_components
    n_features = model.means_.shape[1]
    
    # Number of parameters depends on covariance type
    if cov_type == 'full':
        n_cov_params = n_states * n_features * (n_features + 1) // 2
    elif cov_type == 'diag':
        n_cov_params = n_states * n_features
    elif cov_type == 'tied':
        n_cov_params = n_features * (n_features + 1) // 2
    elif cov_type == 'spherical':
        n_cov_params = n_states
    
    # Calculate total number of parameters
    n_params = n_states - 1  # startprob_
    n_params += n_states * (n_states - 1)  # transmat_
    n_params += n_states * n_features  # means_
    n_params += n_cov_params  # covars_
    
    # Calculate AIC and BIC
    model_score = model.score(reduced_features)
    aic = 2 * n_params - 2 * model_score
    bic = np.log(len(reduced_features)) * n_params - 2 * model_score
    
    print(f"AIC: {aic:.2f} (lower is better)")
    print(f"BIC: {bic:.2f} (lower is better)")
    print(f"Log-likelihood: {model_score:.2f}")
    print(f"Number of parameters: {n_params}")

    
    # Add regime classifications to the training data
    training['Predicted_Regime'] = hidden_states
    training['Regime_Label'] = [regime_labels[s] for s in hidden_states]
    training['Regime_Probability'] = np.max(state_probs, axis=1)
    
    # Add probability columns for each regime
    for i, label in enumerate(regime_labels):
        training[f'Prob_{label}'] = state_probs[:, i]
    
    # Add smoothed probabilities from forward-backward algorithm
    for i, label in enumerate(regime_labels):
        training[f'Smooth_Prob_{label}'] = forwback[:, i]
    
    # Create a bundle of model components for prediction
    model_bundle = {
        'model': model,
        'scaler': scaler,
        'pca': pca,
        'selected_features': selected_features,
        'regime_labels': regime_labels,
        'n_components': n_components
    }
    
    return model_bundle, training, hidden_states, regime_labels, regime_stats

def predict_regimes(model_bundle, data, start_date, end_date):
    """Predict market regimes for a specific date range using the trained HMM model"""
    # Extract model components
    model = model_bundle['model']
    scaler = model_bundle['scaler']
    pca = model_bundle['pca']
    selected_features = model_bundle['selected_features']
    regime_labels = model_bundle['regime_labels']
    
    # Filter data for prediction period
    pred_data = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)].copy()
    
    if len(pred_data) == 0:
        print(f"No data available for period {start_date} to {end_date}")
        return None
    
    # Prepare features for prediction
    feature_data = pred_data[selected_features].values
    
    # Scale features using the same scaler as training
    scaled_features = scaler.transform(feature_data)
    
    # Reduce dimensions using PCA
    reduced_features = pca.transform(scaled_features)
    
    # Predict regimes and probabilities
    hidden_states = model.predict(reduced_features)
    state_probs = model.predict_proba(reduced_features)
    
    # For smoother predictions, use Forward-Backward algorithm
    _, forwback = model.score_samples(reduced_features)
    
    # Add predictions to dataframe
    pred_data['Predicted_Regime'] = hidden_states
    pred_data['Regime_Label'] = [regime_labels[s] for s in hidden_states]
    pred_data['Regime_Probability'] = np.max(state_probs, axis=1)
    
    # Add probability columns for each regime
    for i, label in enumerate(regime_labels):
        pred_data[f'Prob_{label}'] = state_probs[:, i]
        pred_data[f'Smooth_Prob_{label}'] = forwback[:, i]
    
    # Print basic statistics about the prediction
    print(f"Predicted regimes for period {start_date} to {end_date}")
    
    # Calculate regime distribution
    regime_counts = pd.Series(hidden_states).value_counts(normalize=True) * 100
    print("\nRegime Distribution:")
    for regime, percentage in sorted(regime_counts.items()):
        print(f"Regime {regime} [{regime_labels[regime]}]: {percentage:.2f}%")
    
    # Calculate average regime duration
    regime_changes = (pred_data['Predicted_Regime'] != pred_data['Predicted_Regime'].shift(1)).sum()
    avg_duration = len(pred_data) / (regime_changes if regime_changes > 0 else 1)
    print(f"\nRegime persistence: {avg_duration:.2f} days average duration")
    
    return pred_data

def apply_viterbi_smoothing(model, data):
    """
    Apply the Viterbi algorithm to get the most likely state sequence
    This can reduce regime switching compared to standard prediction
    """
    # Get the most likely sequence of hidden states using Viterbi algorithm
    state_sequence = model.predict(data)
    return state_sequence

def apply_transition_constraints(predictions, min_duration=10):
    """
    Post-process regime predictions to enforce minimum duration
    This reduces frequent regime switching
    
    Parameters:
    -----------
    predictions: array of predicted regime states
    min_duration: minimum number of days a regime should persist
    
    Returns:
    --------
    array of smoothed regime states
    """
    smoothed = predictions.copy()
    
    # Find regime transitions
    transitions = np.where(smoothed[1:] != smoothed[:-1])[0] + 1
    
    # Add start and end points to transitions
    transitions = np.insert(transitions, 0, 0)
    transitions = np.append(transitions, len(smoothed))
    
    # Check each segment between transitions
    for i in range(len(transitions) - 1):
        start = transitions[i]
        end = transitions[i+1]
        segment_length = end - start
        
        # If segment is too short, merge with adjacent regimes
        if segment_length < min_duration:
            # Determine which adjacent segment to merge with
            # Choose the one with higher regime probability if available
            if i == 0:  # First segment
                smoothed[start:end] = smoothed[end]
            elif i == len(transitions) - 2:  # Last segment
                smoothed[start:end] = smoothed[start-1]
            else:
                # Check which adjacent segment is longer
                prev_segment_length = start - transitions[i-1]
                next_segment_length = transitions[i+2] - end if i+2 < len(transitions) else float('inf')
                
                if prev_segment_length > next_segment_length:
                    smoothed[start:end] = smoothed[start-1]
                else:
                    smoothed[start:end] = smoothed[end]
    
    return smoothed

def calculate_conditional_metrics(data, regime_column='Regime_Label'):
    """
    Calculate conditional performance metrics for each regime
    
    Parameters:
    -----------
    data: DataFrame with regime classifications and return data
    regime_column: column name containing regime labels
    
    Returns:
    --------
    DataFrame with performance metrics by regime
    """
    metrics = []
    
    for regime in data[regime_column].unique():
        regime_data = data[data[regime_column] == regime]
        
        # Skip if insufficient data
        if len(regime_data) < 10:
            continue
        
        # Calculate daily return statistics
        daily_return_mean = regime_data['Log_Return'].mean()
        daily_return_std = regime_data['Log_Return'].std()
        
        # Calculate hit rate (% of positive days)
        hit_rate = (regime_data['Log_Return'] > 0).mean() * 100
        
        # Calculate maximum drawdown
        regime_data['Cumulative_Return'] = (1 + regime_data['Log_Return'] / 100).cumprod()
        rolling_max = regime_data['Cumulative_Return'].cummax()
        drawdown = (regime_data['Cumulative_Return'] / rolling_max - 1) * 100
        max_drawdown = drawdown.min()
        
        # Calculate annualized metrics
        trading_days_per_year = 252
        annualized_return = daily_return_mean * trading_days_per_year
        annualized_vol = daily_return_std * np.sqrt(trading_days_per_year)
        sharpe_ratio = annualized_return / annualized_vol if annualized_vol > 0 else 0
        
        # Store metrics
        metrics.append({
            'Regime': regime,
            'Count': len(regime_data),
            'Avg_Daily_Return': daily_return_mean,
            'Daily_Std': daily_return_std,
            'Hit_Rate': hit_rate,
            'Max_Drawdown': max_drawdown,
            'Annualized_Return': annualized_return,
            'Annualized_Vol': annualized_vol,
            'Sharpe_Ratio': sharpe_ratio
        })
    
    return pd.DataFrame(metrics)

# ======== VISUALIZATION FUNCTIONS ========
def plot_regimes(results, title=None):
    """Plot SPY price with regime classifications"""
    if results is None or len(results) == 0:
        print("No data available to plot")
        return
    
    # Set plot title
    if title is None:
        start_date = results['Date'].min().strftime('%Y-%m-%d')
        end_date = results['Date'].max().strftime('%Y-%m-%d')
        title = f'Market Regimes from {start_date} to {end_date}'
    
    # Define specific colors for each regime type
    color_map = {
        'Bull': 'green',
        'Bear': 'red',
        'Neutral': 'gold'
    }
    
    # Create figure with two subplots
    fig = make_subplots(rows=3, cols=1, 
                        shared_xaxes=True,
                        vertical_spacing=0.1,
                        row_heights=[0.5, 0.25, 0.25],
                        subplot_titles=(title, "Regime Probabilities", "Regime Performance"))
    
    # Add price line
    fig.add_trace(
        go.Scatter(
            x=results['Date'],
            y=results['Close'],
            mode='lines',
            line=dict(color='rgba(0,0,0,0.3)', width=1),
            name=f'{TICKER} Price'
        ),
        row=1, col=1
    )
    
    # Add colored markers for different regimes
    unique_regimes = results['Regime_Label'].unique()
    for regime in sorted(unique_regimes):
        regime_data = results[results['Regime_Label'] == regime]
        
        # Use our predefined colors based on regime label
        color = color_map.get(regime, 'gray')
        
        fig.add_trace(
            go.Scatter(
                x=regime_data['Date'], 
                y=regime_data['Close'],
                mode='markers',
                marker=dict(color=color, size=6),
                name=f'{regime} Regime',
                hovertemplate='%{x}<br>Price: %{y:.2f}<br>Regime: ' + regime + 
                              '<br>Probability: %{text:.2f}',
                text=regime_data['Regime_Probability']
            ),
            row=1, col=1
        )
    
    # Add regime probability traces in the second subplot
    # Use smoothed probabilities if available
    prob_prefix = 'Smooth_Prob_' if 'Smooth_Prob_Bull' in results.columns else 'Prob_'
    
    for regime in sorted(unique_regimes):
        color = color_map.get(regime, 'gray')
        fig.add_trace(
            go.Scatter(
                x=results['Date'],
                y=results[f'{prob_prefix}{regime}'],
                mode='lines',
                line=dict(width=2, color=color),
                name=f'{regime} Probability'
            ),
            row=2, col=1
        )
    
    # Add a horizontal line at 0.5 probability
    fig.add_shape(
        type="line",
        x0=results['Date'].min(),
        y0=0.5,
        x1=results['Date'].max(),
        y1=0.5,
        line=dict(color="black", width=1, dash="dash"),
        row=2, col=1
    )
    
    # Calculate cumulative returns by regime
    results['Return'] = results['Log_Return'] / 100  # Convert to decimal
    
    # Calculate regime-specific returns
    for regime in sorted(unique_regimes):
        mask = results['Regime_Label'] == regime
        results[f'Return_{regime}'] = results['Return'].copy()
        results.loc[~mask, f'Return_{regime}'] = 0
        
        # Calculate cumulative returns (1 + r)
        results[f'Cumulative_{regime}'] = (1 + results[f'Return_{regime}']).cumprod()
        
        # Add to the third subplot
        color = color_map.get(regime, 'gray')
        fig.add_trace(
            go.Scatter(
                x=results['Date'],
                y=results[f'Cumulative_{regime}'],
                mode='lines',
                line=dict(width=2, color=color),
                name=f'{regime} Performance'
            ),
            row=3, col=1
        )
    
    # Also add the overall buy-and-hold performance
    results['Cumulative_All'] = (1 + results['Return']).cumprod()
    fig.add_trace(
        go.Scatter(
            x=results['Date'],
            y=results['Cumulative_All'],
            mode='lines',
            line=dict(width=2, color='black', dash='dash'),
            name='Buy & Hold'
        ),
        row=3, col=1
    )
    
    # Update layout
    fig.update_layout(
        xaxis_title='Date',
        yaxis_title=f'{TICKER} Price',
        yaxis2_title='Probability',
        yaxis3_title='Growth of $1',
        template='plotly_white',
        legend_title='Market Regimes',
        hovermode='closest',
        height=1000
    )
    
    # Set y-axis range for probability subplot
    fig.update_yaxes(range=[0, 1], row=2, col=1)
    
    fig.show()
    
    # Create pie chart showing regime distribution
    regime_distribution = results['Regime_Label'].value_counts().reset_index()
    regime_distribution.columns = ['Regime', 'Days']
    regime_distribution['Percentage'] = regime_distribution['Days'] / len(results) * 100
    
    # Calculate performance metrics by regime
    metrics = calculate_conditional_metrics(results)
    
    # Print regime performance metrics
    print("\nRegime Performance Metrics:")
    print("=" * 100)
    print(f"{'Regime':<10} {'Days':<10} {'Annualized Return':<20} {'Annualized Vol':<20} {'Sharpe':<10} {'Hit Rate':<10} {'Max DD':<10}")
    print("-" * 100)
    
    for _, row in metrics.iterrows():
        print(f"{row['Regime']:<10} {row['Count']:<10.0f} {row['Annualized_Return']:<20.2f} "
              f"{row['Annualized_Vol']:<20.2f} {row['Sharpe_Ratio']:<10.2f} {row['Hit_Rate']:<10.2f} {row['Max_Drawdown']:<10.2f}")
    
    # Create performance heatmap
    fig_perf = px.bar(
        metrics, 
        x='Regime', 
        y='Annualized_Return',
        color='Regime',
        color_discrete_map=color_map,
        title='Annualized Returns by Regime',
        text='Sharpe_Ratio',
        labels={'Annualized_Return': 'Annualized Return (%)', 'Regime': 'Market Regime'}
    )
    
    fig_perf.update_layout(
        yaxis_title='Annualized Return (%)',
        xaxis_title='Market Regime',
        template='plotly_white',
        height=500
    )
    
    fig_perf.show()
    
    # Create regime duration chart
    regime_durations = []
    current_regime = None
    current_start = None
    
    # Calculate regime duration periods
    for i, row in results.iterrows():
        if current_regime is None:
            current_regime = row['Regime_Label']
            current_start = row['Date']
        elif row['Regime_Label'] != current_regime:
            # Regime change detected
            end_date = row['Date']
            duration = (end_date - current_start).days
            
            regime_durations.append({
                'Regime': current_regime,
                'Start': current_start,
                'End': end_date,
                'Duration': duration
            })
            
            current_regime = row['Regime_Label']
            current_start = row['Date']
    
    # Add the last regime period
    if current_regime is not None:
        end_date = results['Date'].iloc[-1]
        duration = (end_date - current_start).days
        
        regime_durations.append({
            'Regime': current_regime,
            'Start': current_start,
            'End': end_date,
            'Duration': duration
        })
    
    # Convert to DataFrame
    regime_periods = pd.DataFrame(regime_durations)
    
    # Print regime periods
    if len(regime_periods) > 0:
        print("\nRegime Periods:")
        print("=" * 80)
        print(f"{'Regime':<10} {'Start':<12} {'End':<12} {'Duration (days)':<15}")
        print("-" * 80)
        
        for _, row in regime_periods.iterrows():
            print(f"{row['Regime']:<10} {row['Start'].strftime('%Y-%m-%d'):<12} {row['End'].strftime('%Y-%m-%d'):<12} {row['Duration']:<15.0f}")
    
    return metrics, regime_periods

def plot_viterbi_comparison(original_states, viterbi_states, dates, close_prices, regime_labels):
    """Compare original predictions with Viterbi algorithm smoothed predictions"""
    
    fig = make_subplots(rows=2, cols=1, 
                       shared_xaxes=True,
                       vertical_spacing=0.1,
                       row_heights=[0.7, 0.3],
                       subplot_titles=('Original vs. Viterbi Smoothed Regimes', 'Regime Changes'))
    
    # Define colors for regimes
    color_map = {
        'Bull': 'green',
        'Bear': 'red',
        'Neutral': 'gold'
    }
    
    # Add price line
    fig.add_trace(
        go.Scatter(
            x=dates,
            y=close_prices,
            mode='lines',
            line=dict(color='rgba(0,0,0,0.3)', width=1),
            name='Price'
        ),
        row=1, col=1
    )
    
    # Add original states
    original_labels = [regime_labels[s] for s in original_states]
    for regime in sorted(set(original_labels)):
        mask = np.array(original_labels) == regime
        
        fig.add_trace(
            go.Scatter(
                x=dates[mask],
                y=close_prices[mask],
                mode='markers',
                marker=dict(color=color_map.get(regime, 'gray'), size=8, symbol='circle'),
                name=f'Original {regime}'
            ),
            row=1, col=1
        )
    
    # Add viterbi states with different marker
    viterbi_labels = [regime_labels[s] for s in viterbi_states]
    for regime in sorted(set(viterbi_labels)):
        mask = np.array(viterbi_labels) == regime
        
        fig.add_trace(
            go.Scatter(
                x=dates[mask],
                y=close_prices[mask],
                mode='markers',
                marker=dict(color=color_map.get(regime, 'gray'), size=4, symbol='x'),
                name=f'Viterbi {regime}'
            ),
            row=1, col=1
        )
    
    # Add a trace showing where states differ
    differ_mask = original_states != viterbi_states
    
    if np.any(differ_mask):
        fig.add_trace(
            go.Scatter(
                x=dates[differ_mask],
                y=close_prices[differ_mask],
                mode='markers',
                marker=dict(color='purple', size=10, symbol='star'),
                name='States Differ'
            ),
            row=1, col=1
        )
    
    # Add regime change indicators
    original_changes = np.diff(original_states, prepend=original_states[0]) != 0
    viterbi_changes = np.diff(viterbi_states, prepend=viterbi_states[0]) != 0
    
    fig.add_trace(
        go.Scatter(
            x=dates,
            y=original_changes.astype(int),
            mode='lines',
            line=dict(color='blue', width=2),
            name='Original Changes'
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=dates,
            y=viterbi_changes.astype(int),
            mode='lines',
            line=dict(color='red', width=2),
            name='Viterbi Changes'
        ),
        row=2, col=1
    )
    
    # Update layout
    fig.update_layout(
        title='Comparison of Original vs. Viterbi Smoothed Regimes',
        xaxis_title='Date',
        yaxis_title='Price',
        template='plotly_white',
        height=800
    )
    
    fig.update_yaxes(title='Regime Change', range=[-0.1, 1.1], row=2, col=1)
    
    fig.show()
    
    # Print statistics about the differences
    total_differ = np.sum(differ_mask)
    original_change_count = np.sum(original_changes)
    viterbi_change_count = np.sum(viterbi_changes)
    
    print(f"State Difference Statistics:")
    print(f"Total days with different regimes: {total_differ} ({total_differ/len(dates)*100:.2f}%)")
    print(f"Original regime changes: {original_change_count}")
    print(f"Viterbi regime changes: {viterbi_change_count}")
    print(f"Reduction in regime changes: {original_change_count - viterbi_change_count} ({(original_change_count - viterbi_change_count)/original_change_count*100:.2f}%)")
    
    return total_differ, original_change_count, viterbi_change_count

def plot_feature_importance(model_bundle, training_data):
    """
    Plot feature importance for the HMM model based on component means
    """
    try:
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        model = model_bundle['model']
        selected_features = model_bundle['selected_features']
        regime_labels = model_bundle['regime_labels']
        pca = model_bundle['pca']
        
        # Get means from the model
        means = model.means_
        
        # If we're using PCA, we need to transform means back to original feature space
        n_components = pca.n_components_
        
        # Create a more interpretable feature importance visualization
        # 1. Get PCA component loadings
        component_loadings = pca.components_.T
        
        # Print the PCA loadings to help interpret components
        loading_df = pd.DataFrame(
            component_loadings,
            index=selected_features,
            columns=[f"PC{i+1}" for i in range(n_components)]
        )
        
        print("PCA Component Loadings:")
        print(loading_df)
        
        # 2. Plot a heatmap of means in PCA space
        plt.figure(figsize=(12, 6))
        
        sns.heatmap(means, cmap='RdBu_r', center=0,
                   xticklabels=[f"PC{i+1}" for i in range(n_components)],
                   yticklabels=[f"Regime {i} ({regime_labels[i]})" for i in range(len(regime_labels))],
                   annot=True, fmt=".2f")
        
        plt.title('Regime Means in PCA Space')
        plt.tight_layout()
        plt.show()
        
        # 3. Generate approximate feature importance by combining PCA loadings with means
        feature_importance = {}
        
        for i, feature in enumerate(selected_features):
            importance = 0
            for j in range(n_components):
                importance += abs(component_loadings[i, j]) * np.std(means[:, j])
            feature_importance[feature] = importance
        
        # Sort and plot feature importance
        sorted_importance = {k: v for k, v in sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)}
        
        plt.figure(figsize=(14, 6))
        plt.bar(sorted_importance.keys(), sorted_importance.values())
        plt.xticks(rotation=90)
        plt.title('Feature Importance (PCA-derived)')
        plt.tight_layout()
        plt.show()
        
        # 4. Plot the state space for the first two principal components
        plt.figure(figsize=(10, 8))
        
        # Extract feature data
        feature_data = training_data[selected_features].values
        scaler = model_bundle['scaler']
        scaled_features = scaler.transform(feature_data)
        
        # Reduce to first two PCA components
        reduced_features = pca.transform(scaled_features)
        
        # Get regime predictions for each data point
        hidden_states = model.predict(reduced_features)
        
        # Plot data points colored by predicted regime
        for i, label in enumerate(regime_labels):
            mask = hidden_states == i
            plt.scatter(
                reduced_features[mask, 0],
                reduced_features[mask, 1],
                alpha=0.5,
                label=label,
                color=color_map.get(label, 'gray')
            )
        
        # Plot regime centers
        plt.scatter(
            means[:, 0],
            means[:, 1],
            marker='*',
            s=300,
            edgecolor='k',
            label='Regime Centers',
            c=[color_map.get(label, 'gray') for label in regime_labels]
        )
        
        # Add arrows for the top features
        top_features = list(sorted_importance.keys())[:5]  # Top 5 features
        top_indices = [selected_features.index(f) for f in top_features]
        
        for i in top_indices:
            plt.arrow(
                0, 0,
                component_loadings[i, 0] * 3,
                component_loadings[i, 1] * 3,
                head_width=0.1,
                head_length=0.1,
                fc='black',
                ec='black'
            )
            plt.text(
                component_loadings[i, 0] * 3.1,
                component_loadings[i, 1] * 3.1,
                selected_features[i],
                fontsize=12
            )
        
        plt.xlabel(f'PC1 (Explains {pca.explained_variance_ratio_[0]:.1%} of variance)')
        plt.ylabel(f'PC2 (Explains {pca.explained_variance_ratio_[1]:.1%} of variance)')
        plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
        plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
        plt.title('HMM Regimes in PCA Space with Key Features')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
    except ImportError:
        print("Visualization requires matplotlib and seaborn. Install with: pip install matplotlib seaborn")

# ======== HYBRID APPROACH FUNCTIONS ========
def train_hybrid_hmm_gmm(data, start_date, end_date, n_components=3):
    """
    Hybrid approach combining GMM for initial clustering and HMM for temporal dynamics
    
    This two-stage approach often performs better than either model alone
    """
    from sklearn.mixture import GaussianMixture
    
    # Filter data to training period
    training = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)].copy()
    
    print(f"Training hybrid GMM+HMM model on data from {start_date} to {end_date}")
    
    # 1. Feature preparation - same as in HMM approach
    reduced_features, scaler, pca, selected_features = select_and_prepare_features(
        training, n_components=4
    )
    
    # 2. First stage: GMM for initial clustering
    gmm = GaussianMixture(
        n_components=n_components,
        covariance_type='full',
        max_iter=1000, 
        n_init=20,
        random_state=42
    )
    
    # Fit GMM
    gmm.fit(reduced_features)
    
    # Get cluster assignments and probabilities
    gmm_labels = gmm.predict(reduced_features)
    gmm_probs = gmm.predict_proba(reduced_features)
    
    # Analyze GMM cluster properties to establish mapping to regimes
    regime_stats = {}
    for i in range(n_components):
        cluster_mask = (gmm_labels == i)
        if np.sum(cluster_mask) > 0:
            regime_stats[i] = {
                'count': np.sum(cluster_mask),
                'return_avg': np.mean(training.loc[cluster_mask, 'Log_Return']),
                'volatility_avg': np.mean(training.loc[cluster_mask, 'Volatility']),
                'vix_avg': np.mean(training.loc[cluster_mask, 'VIX']),
                'momentum_avg': np.mean(training.loc[cluster_mask, 'Momentum'])
            }
    
    # Assign regime labels based on average returns
    return_avgs = [(i, regime_stats[i]['return_avg']) for i in regime_stats.keys()]
    return_avgs.sort(key=lambda x: x[1], reverse=True)
    
    gmm_mapping = {}
    regime_labels = ['Bull', 'Neutral', 'Bear']
    
    for idx, (cluster, _) in enumerate(return_avgs):
        if idx == 0:
            gmm_mapping[cluster] = 0  # Bull
        elif idx == len(return_avgs) - 1:
            gmm_mapping[cluster] = 2  # Bear
        else:
            gmm_mapping[cluster] = 1  # Neutral
    
    # Use GMM results to initialize HMM
    # 3. Initialize HMM with GMM results
    # Convert GMM clusters to HMM-friendly mapping
    mapped_labels = np.array([gmm_mapping[label] for label in gmm_labels])
    
    # Calculate empirical transition matrix from GMM labels
    trans_mat = np.zeros((n_components, n_components))
    
    for i in range(1, len(mapped_labels)):
        prev_state = mapped_labels[i-1]
        curr_state = mapped_labels[i]
        trans_mat[prev_state, curr_state] += 1
    
    # Convert to probabilities and ensure no zero probabilities
    for i in range(n_components):
        row_sum = trans_mat[i, :].sum()
        if row_sum > 0:
            trans_mat[i, :] = trans_mat[i, :] / row_sum
        else:
            # If no transitions observed, use reasonable defaults
            trans_mat[i, :] = 0.1 / (n_components - 1)
            trans_mat[i, i] = 0.9
    
    # Create HMM model
    model = hmm.GaussianHMM(
        n_components=n_components,
        covariance_type='full',
        n_iter=1000,
        tol=1e-6,
        init_params="",  # Don't initialize params, we'll set them manually
        random_state=42
    )
    
    # Set initial parameters from GMM results
    model.startprob_ = np.array([np.mean(mapped_labels == i) for i in range(n_components)])
    model.transmat_ = trans_mat
    
    # Use GMM means and covariances for HMM initialization
    # Reorder to match our mapping
    n_features = reduced_features.shape[1]
    model.means_ = np.zeros((n_components, n_features))
    
    # Initialize covariances properly according to the covariance type
    if model.covariance_type == 'full':
        # For 'full' covariance type, we need to set _covars_ directly
        # with the correct shape (n_components, n_features, n_features)
        covars = np.stack([np.eye(n_features) for _ in range(n_components)])
        model._covars_ = covars
    elif model.covariance_type == 'diag':
        # For 'diag' covariance type, shape should be (n_components, n_features)
        model._covars_ = np.ones((n_components, n_features))
    elif model.covariance_type == 'tied':
        # For 'tied' covariance type, shape should be (n_features, n_features)
        model._covars_ = np.eye(n_features)
    elif model.covariance_type == 'spherical':
        # For 'spherical' covariance type, shape should be (n_components,)
        model._covars_ = np.ones(n_components)
    
    # Map GMM means to HMM states
    for gmm_idx, hmm_idx in gmm_mapping.items():
        model.means_[hmm_idx] = gmm.means_[gmm_idx]
        
        # Carefully update covariances based on type
        if model.covariance_type == 'full':
            # Ensure covariance matrix is symmetric and positive-definite
            cov = gmm.covariances_[gmm_idx]
            # Make it symmetric
            cov = (cov + cov.T) / 2
            # Ensure positive-definiteness by adding small value to diagonal
            cov += np.eye(cov.shape[0]) * 1e-3
            model._covars_[hmm_idx] = cov
        elif model.covariance_type == 'diag':
            # Extract diagonal elements for diagonal covariance
            model._covars_[hmm_idx] = np.diag(gmm.covariances_[gmm_idx])
        elif model.covariance_type == 'tied' and hmm_idx == 0:  # Only once for tied
            # Average of all covariances for tied covariance
            cov = np.zeros_like(model._covars_)
            for i in range(n_components):
                cov += gmm.covariances_[i]
            cov /= n_components
            # Make it symmetric
            cov = (cov + cov.T) / 2
            # Ensure positive-definiteness
            cov += np.eye(cov.shape[0]) * 1e-3
            model._covars_ = cov
        elif model.covariance_type == 'spherical':
            # Use average of diagonal for spherical
            model._covars_[hmm_idx] = np.mean(np.diag(gmm.covariances_[gmm_idx]))
    
    # 4. Train HMM with good initialization
    model.fit(reduced_features)
    
    # Get state sequence and probabilities
    hidden_states = model.predict(reduced_features)
    state_probs = model.predict_proba(reduced_features)
    
    # Extract smoothed probabilities using Forward-Backward
    _, forwback = model.score_samples(reduced_features)
    
    # Add results to the training data
    training['Predicted_Regime'] = hidden_states
    training['Regime_Label'] = [regime_labels[s] for s in hidden_states]
    training['Regime_Probability'] = np.max(state_probs, axis=1)
    
    # Add probability columns
    for i, label in enumerate(regime_labels):
        training[f'Prob_{label}'] = state_probs[:, i]
        training[f'Smooth_Prob_{label}'] = forwback[:, i]
    
    # Add GMM results for comparison
    training['GMM_Cluster'] = gmm_labels
    training['GMM_Mapping'] = [gmm_mapping[c] for c in gmm_labels]
    training['GMM_Label'] = [regime_labels[gmm_mapping[c]] for c in gmm_labels]
    
    # Calculate regime statistics
    regime_stats = {}
    for i in range(n_components):
        regime_mask = (hidden_states == i)
        if np.sum(regime_mask) > 0:
            regime_stats[i] = {
                'count': np.sum(regime_mask),
                'return_avg': np.mean(training.loc[regime_mask, 'Log_Return']),
                'return_std': np.std(training.loc[regime_mask, 'Log_Return']),
                'volatility_avg': np.mean(training.loc[regime_mask, 'Volatility']),
                'vix_avg': np.mean(training.loc[regime_mask, 'VIX']),
                'vix_ratio_avg': np.mean(training.loc[regime_mask, 'VIX_Ratio']),
                'momentum_avg': np.mean(training.loc[regime_mask, 'Momentum']),
                'prob_avg': np.mean(state_probs[:, i])
            }
    
    # Calculate model evaluation metrics
    # Calculate number of parameters for the HMM
    n_states = model.n_components
    n_features = model.means_.shape[1]
    
    # Number of parameters depends on covariance type
    if model.covariance_type == 'full':
        n_cov_params = n_states * n_features * (n_features + 1) // 2
    elif model.covariance_type == 'diag':
        n_cov_params = n_states * n_features
    elif model.covariance_type == 'tied':
        n_cov_params = n_features * (n_features + 1) // 2
    elif model.covariance_type == 'spherical':
        n_cov_params = n_states
    
    # Calculate total number of parameters
    n_params = n_states - 1  # startprob_
    n_params += n_states * (n_states - 1)  # transmat_
    n_params += n_states * n_features  # means_
    n_params += n_cov_params  # covars_
    
    # Create model bundle
    model_bundle = {
        'model': model,
        'gmm': gmm,
        'gmm_mapping': gmm_mapping,
        'scaler': scaler,
        'pca': pca,
        'selected_features': selected_features,
        'regime_labels': regime_labels,
        'n_components': n_components
    }
    
    # Print key regime statistics
    print("\nHybrid Model - Regime Characteristics:")
    print("=" * 80)
    print(f"{'Regime':<8} {'Label':<8} {'Count':<8} {'Return %':<10} {'Vol':<8} {'VIX':<8} {'Momentum':<10}")
    print("-" * 80)
    
    for i in range(n_components):
        if i in regime_stats:
            stats = regime_stats[i]
            print(f"{i:<8} {regime_labels[i]:<8} {stats['count']:<8} "
                  f"{stats['return_avg']:<10.2f} {stats['volatility_avg']:<8.2f} {stats['vix_avg']:<8.2f} "
                  f"{stats['momentum_avg']:<10.2f}")
    
    # Print transition matrix
    print("\nRegime Transition Matrix:")
    print("=" * 60)
    print(f"{'From/To':<10}", end="")
    for i in range(n_components):
        print(f"{regime_labels[i]:<10}", end="")
    print()
    print("-" * 60)
    
    for i in range(n_components):
        print(f"{regime_labels[i]:<10}", end="")
        for j in range(n_components):
            print(f"{model.transmat_[i, j]:<10.4f}", end="")
        print()
    
    # Compare GMM and HMM results
    agreement = np.mean(training['GMM_Mapping'] == training['Predicted_Regime']) * 100
    print(f"\nGMM and HMM agreement: {agreement:.2f}%")
    
    # Calculate average regime duration
    regime_changes = (training['Predicted_Regime'] != training['Predicted_Regime'].shift(1)).sum()
    avg_duration = len(training) / (regime_changes if regime_changes > 0 else 1)
    print(f"HMM average regime duration: {avg_duration:.2f} days")
    
    gmm_changes = (training['GMM_Mapping'] != training['GMM_Mapping'].shift(1)).sum()
    gmm_avg_duration = len(training) / (gmm_changes if gmm_changes > 0 else 1)
    print(f"GMM average regime duration: {gmm_avg_duration:.2f} days")
    
    return model_bundle, training, hidden_states, regime_labels, regime_stats

# ======== MAIN EXECUTION ========
def main():
    """Main execution function with model comparison"""
    # Download and prepare market data
    df = download_market_data(
        TICKER, VIX_TICKER, TNX_TICKER, GLD_TICKER, 
        XLY_TICKER, XLP_TICKER, XLU_TICKER, XLF_TICKER,
        HYG_TICKER, TLT_TICKER, START_DATE
    )
    
    # Load and prepare A/D line data
    ad_data = load_ad_line_data("nyse_breadth_2023.csv")
    
    # Calculate features including A/D line indicators
    df = calculate_features(df, ad_data)
    
    # 1. Train basic HMM model
    hmm_model, hmm_training, hmm_states, regime_labels, _ = train_hmm_model(
        df, TRAIN_START_DATE, TRAIN_END_DATE, NUM_REGIMES, MAX_ITERATIONS, COVARIANCE_TYPE
    )
    
    # Plot training period results for HMM
    print("\nHMM Model - Training Period Results:")
    plot_regimes(hmm_training, f'HMM Market Regimes - Training Period ({TRAIN_START_DATE} to {TRAIN_END_DATE})')
    
    # 2. Train hybrid HMM+GMM model
    hybrid_model, hybrid_training, hybrid_states, _, _ = train_hybrid_hmm_gmm(
        df, TRAIN_START_DATE, TRAIN_END_DATE, NUM_REGIMES
    )
    
    # Plot training period results for hybrid model
    print("\nHybrid Model - Training Period Results:")
    plot_regimes(hybrid_training, f'Hybrid GMM+HMM Market Regimes - Training Period ({TRAIN_START_DATE} to {TRAIN_END_DATE})')
    
    # 3. Test on recent period
    recent_start = '2025-01-02'
    recent_end = (datetime.today() + timedelta(days=1)).strftime("%Y-%m-%d")
    #recent_end ='2025-03-03'
    
    # Predict with HMM
    print(f"\nPredicting regimes for recent period ({recent_start} to {recent_end}) with HMM model:")
    hmm_recent = predict_regimes(hmm_model, df, recent_start, recent_end)
    
    # Predict with hybrid model
    print(f"\nPredicting regimes for recent period ({recent_start} to {recent_end}) with hybrid model:")
    hybrid_recent = predict_regimes(hybrid_model, df, recent_start, recent_end)
    
    # Plot recent period results for both models
    if hmm_recent is not None:
        plot_regimes(hmm_recent, f'HMM Market Regimes - Recent Period ({recent_start} to {recent_end})')
    
    if hybrid_recent is not None:
        plot_regimes(hybrid_recent, f'Hybrid GMM+HMM Market Regimes - Recent Period ({recent_start} to {recent_end})')
    
    # 4. Compare different smoothing approaches for HMM
    if hmm_recent is not None:
        # Get original predictions
        orig_states = hmm_recent['Predicted_Regime'].values
        
        # Apply Viterbi algorithm for global path optimization
        reduced_features = hmm_model['pca'].transform(
            hmm_model['scaler'].transform(hmm_recent[hmm_model['selected_features']].values)
        )
        viterbi_states = apply_viterbi_smoothing(hmm_model['model'], reduced_features)
        
        # Apply transition constraints for minimum duration
        constrained_states = apply_transition_constraints(orig_states, min_duration=10)
        
        # Compare results
        plot_viterbi_comparison(
            orig_states, viterbi_states, 
            hmm_recent['Date'].values, hmm_recent['Close'].values,
            hmm_model['regime_labels']
        )
        
        # Compare duration-constrained to original
        plot_viterbi_comparison(
            orig_states, constrained_states,
            hmm_recent['Date'].values, hmm_recent['Close'].values,
            hmm_model['regime_labels']
        )
    
    # 5. Create performance report and analysis
    print("\nModel Comparison Performance Report:")
    print("=" * 100)
    
    if hmm_recent is not None and hybrid_recent is not None:
        # Calculate performance metrics
        hmm_metrics = calculate_conditional_metrics(hmm_recent)
        hybrid_metrics = calculate_conditional_metrics(hybrid_recent)
        
        # Compare models on key metrics
        print("\nHMM Model Regime Performance:")
        for _, row in hmm_metrics.iterrows():
            print(f"{row['Regime']:<10} Annual Return: {row['Annualized_Return']:.2f}%, "
                  f"Sharpe: {row['Sharpe_Ratio']:.2f}, Hit Rate: {row['Hit_Rate']:.2f}%")
        
        print("\nHybrid Model Regime Performance:")
        for _, row in hybrid_metrics.iterrows():
            print(f"{row['Regime']:<10} Annual Return: {row['Annualized_Return']:.2f}%, "
                  f"Sharpe: {row['Sharpe_Ratio']:.2f}, Hit Rate: {row['Hit_Rate']:.2f}%")
        
        # Calculate disagreement between models
        merged = pd.merge(
            hmm_recent[['Date', 'Regime_Label']], 
            hybrid_recent[['Date', 'Regime_Label']], 
            on='Date', suffixes=('_HMM', '_Hybrid')
        )
        
        disagreement = (merged['Regime_Label_HMM'] != merged['Regime_Label_Hybrid']).mean() * 100
        print(f"\nModel Disagreement: {disagreement:.2f}% of days")
        
        # Create a confusion matrix between models
        from sklearn.metrics import confusion_matrix
        
        labels = sorted(list(set(regime_labels)))
        conf_matrix = confusion_matrix(
            merged['Regime_Label_HMM'], 
            merged['Regime_Label_Hybrid'],
            labels=labels
        )
        
        print("\nConfusion Matrix (HMM vs Hybrid):")
        print(f"{'HMM Hybrid':<15}", end="")
        for label in labels:
            print(f"{label:<10}", end="")
        print()
        
        for i, label in enumerate(labels):
            print(f"{label:<15}", end="")
            for j in range(len(labels)):
                print(f"{conf_matrix[i, j]:<10}", end="")
            print()
    
    # 6. Apply model to create example trading strategy
    if hmm_recent is not None:
        # Simple regime-based allocation
        # Bull: 100% SPY
        # Neutral: 50% SPY, 50% Cash
        # Bear: 30% SPY, 70% Cash/Bond/Gold
        
        hmm_recent['Strategy_Allocation'] = hmm_recent['Regime_Label'].map({
            'Bull': 1.0,    # 100% SPY
            'Neutral': 0.5, # 50% SPY
            'Bear': 0.3     # 30% SPY
        })
        
        # Calculate strategy returns
        hmm_recent['Strategy_Return'] = hmm_recent['Strategy_Allocation'] * hmm_recent['Log_Return'] / 100
        
        # Calculate cumulative returns
        hmm_recent['SPY_Cumulative'] = (1 + hmm_recent['Log_Return'] / 100).cumprod()
        hmm_recent['Strategy_Cumulative'] = (1 + hmm_recent['Strategy_Return']).cumprod()
        
        # Calculate performance metrics
        spy_annual_return = np.mean(hmm_recent['Log_Return']) * 252  # Annualized
        spy_annual_vol = np.std(hmm_recent['Log_Return']) * np.sqrt(252)
        spy_sharpe = spy_annual_return / spy_annual_vol if spy_annual_vol > 0 else 0
        
        strat_annual_return = np.mean(hmm_recent['Strategy_Return'] * 100) * 252  # Annualized
        strat_annual_vol = np.std(hmm_recent['Strategy_Return'] * 100) * np.sqrt(252)
        strat_sharpe = strat_annual_return / strat_annual_vol if strat_annual_vol > 0 else 0
        
        # Plot strategy performance
        fig = go.Figure()
        
        fig.add_trace(
            go.Scatter(
                x=hmm_recent['Date'],
                y=hmm_recent['SPY_Cumulative'],
                mode='lines',
                name='SPY Buy & Hold',
                line=dict(color='blue')
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=hmm_recent['Date'],
                y=hmm_recent['Strategy_Cumulative'],
                mode='lines',
                name='HMM Regime Strategy',
                line=dict(color='green')
            )
        )
        
        # Add regime background
        for regime in sorted(hmm_recent['Regime_Label'].unique()):
            regime_data = hmm_recent[hmm_recent['Regime_Label'] == regime]
            if len(regime_data) > 0:
                for i in range(len(regime_data) - 1):
                    color = {'Bull': 'rgba(0,255,0,0.1)', 'Neutral': 'rgba(255,255,0,0.1)', 'Bear': 'rgba(255,0,0,0.1)'}
                    
                    fig.add_shape(
                        type="rect",
                        x0=regime_data['Date'].iloc[i],
                        y0=0,
                        x1=regime_data['Date'].iloc[i+1],
                        y1=max(hmm_recent['SPY_Cumulative'].max(), hmm_recent['Strategy_Cumulative'].max()) * 1.1,
                        line=dict(width=0),
                        fillcolor=color.get(regime, 'rgba(0,0,0,0.1)')
                    )
        
        fig.update_layout(
            title=f'HMM Regime-Based Strategy Performance ({recent_start} to {recent_end})',
            xaxis_title='Date',
            yaxis_title='Growth of $1',
            legend_title='Strategy',
            template='plotly_white',
            height=600,
            annotations=[
                dict(
                    x=0.02, y=0.98, xref="paper", yref="paper",
                    text=f"SPY: {spy_annual_return:.2f}% Ann. Return, {spy_sharpe:.2f} Sharpe",
                    showarrow=False, align="left", bgcolor="rgba(255,255,255,0.8)"
                ),
                dict(
                    x=0.02, y=0.93, xref="paper", yref="paper",
                    text=f"Strategy: {strat_annual_return:.2f}% Ann. Return, {strat_sharpe:.2f} Sharpe",
                    showarrow=False, align="left", bgcolor="rgba(255,255,255,0.8)"
                )
            ]
        )
        
        fig.show()
        
        # Print strategy performance
        print(f"\nStrategy Performance ({recent_start} to {recent_end}):")
        print(f"SPY Buy & Hold: {spy_annual_return:.2f}% annual return, {spy_annual_vol:.2f}% volatility, {spy_sharpe:.2f} Sharpe")
        print(f"HMM Regime Strategy: {strat_annual_return:.2f}% annual return, {strat_annual_vol:.2f}% volatility, {strat_sharpe:.2f} Sharpe")
        
        # Calculate max drawdowns
        def calculate_max_drawdown(returns):
            cumulative = (1 + returns).cumprod()
            running_max = cumulative.cummax()
            drawdown = (cumulative / running_max - 1) * 100
            return drawdown.min()
        
        spy_drawdown = calculate_max_drawdown(hmm_recent['Log_Return'] / 100)
        strat_drawdown = calculate_max_drawdown(hmm_recent['Strategy_Return'])
        
        print(f"SPY Maximum Drawdown: {spy_drawdown:.2f}%")
        print(f"Strategy Maximum Drawdown: {strat_drawdown:.2f}%")
    
    # Return all models and data for further analysis
    return {
        'hmm_model': hmm_model,
        'hybrid_model': hybrid_model,
        'data': df,
        'ad_data': ad_data,
        'hmm_training': hmm_training,
        'hybrid_training': hybrid_training,
        'hmm_recent': hmm_recent if 'hmm_recent' in locals() else None,
        'hybrid_recent': hybrid_recent if 'hybrid_recent' in locals() else None
    }

if __name__ == "__main__":
    model_objects = main()
